In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
date = "2024-12-30"

In [0]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import sum, desc, rank, asc, lit
from pyspark.sql.window import Window

In [0]:
movies_df = spark.read \
    .parquet(f"{silver_folder_path}/movies") \
    .filter(f"file_date = '{date}' ")

In [0]:
genres_df = spark.read \
    .parquet(f"{silver_folder_path}/genres")

In [0]:
movies_genres_df = spark.read \
    .parquet(f"{silver_folder_path}/movies_genres") \
    .filter(f"file_date = '{date}' ")

In [0]:
genres_mov_gen_df = genres_df.join(movies_genres_df,
                                   genres_df.genre_id == movies_genres_df.genre_id,
                                   "inner") \
                    .select(genres_df.genre_name,
                            movies_genres_df.movie_id)

In [0]:
movies_filter_df = movies_df.filter("year_release_date = 2015")

In [0]:
results_movies_genres_df = movies_filter_df.join(genres_mov_gen_df,
                                                 movies_filter_df.movie_id == genres_mov_gen_df.movie_id,
                                                 "inner")

In [0]:
results_df = results_movies_genres_df.select("year_release_date", "genre_name", "budget", "revenue") \
    .withColumn("revenue", results_movies_genres_df["revenue"].cast(IntegerType()))

In [0]:
results_group_by_df = results_df.groupBy("year_release_date", "genre_name") \
                    .agg(
                        sum("budget").alias("total_budget"),
                        sum("revenue").alias("total_revenue")
                    )

In [0]:
rank_function = Window.partitionBy("year_release_date").orderBy(desc("total_budget"), desc("total_revenue"))

In [0]:

final_df = results_group_by_df.withColumn("rank", rank().over(rank_function)) \
                            .withColumn("created_date", lit(date))

In [0]:
#final_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_group_movie_genre")

In [0]:
final_df.write.mode("overwrite").format("parquet").partitionBy("created_date").saveAsTable("movie_gold.results_group_movie_genre")

In [0]:
%sql
select *
from movie_gold.results_group_movie_genre